In [ ]:
import torch
import numpy as np
import random
import evaluate
import pandas as pd
import textwrap
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from datasets import load_dataset
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, get_scheduler

import ipywidgets as widgets
from IPython.display import display

Configuracion para la Reproducibilidad

In [ ]:
# Seed para asegurar reproducibilidad sin importar el dispositivo en el que se ejecute
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

### Carga y Preprocesamiento del Dataset

In [ ]:
# Cargar el dataset
dataset = load_dataset("mteb/stsbenchmark-sts")

# Normalizar las etiquetas (de 0-5 a un rango de 0-1)
def normalize_labels(examples):
    examples["score"] = [s / 5.0 for s in examples["score"]]
    return examples

# Aplicar normalización
dataset = dataset.map(normalize_labels, batched=True)x

Tokenizacion del Texto

In [ ]:
#Creamos un tokenizador para convertir las frases en IDs de tokens que el modelo pueda procesar. Usare BERT o un modelo optimizado para similitud semántica
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

#Definiendo la funcion de tokenizacion
def tokenize_function(examples):
    return tokenizer(
        examples['sentence1'],
        examples['sentence2'],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    
#Tokenizando el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Creando la Funcion para Fine-Tuning en los Modelos